In [ ]:
# | default_exp routes.datacenter

In [ ]:
# | exporti
from enum import Enum
from typing import Union
import httpx

import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoError as de
import domolibrary.client.DomoAuth as dmda

In [ ]:
# | export
class Datacenter_Enum(Enum):
    ACCOUNT = "ACCOUNT"
    CARD = "CARD"
    DATAFLOW = "DATAFLOW"
    DATASET = "DATASET"
    GROUP = "GROUP"
    PAGE = "PAGE"
    USER = "USER"


class Dataflow_Type_Filter_Enum(Enum):
    ADR = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "ADR",
        "name": "ADR",
        "not": False,
    }

    MYSQL = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MYSQL",
        "name": "MYSQL",
        "not": False,
    }

    REDSHIFT = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MYSQL",
        "name": "MYSQL",
        "not": False,
    }

    MAGICV2 = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "MAGIC",
        "name": "Magic ETL v2",
        "not": False,
    }

    MAGIC = {
        "filterType": "term",
        "field": "data_flow_type",
        "value": "ETL",
        "name": "Magic ETL",
        "not": False,
    }

In [ ]:
# | export


class Datacenter_Filter_Field_Enum(Enum):
    DATAPROVIDER = "dataprovidername_facet"
    CERTIFICATION = "certification.state"


class Datacenter_Filter_Field_Certification_Enum(Enum):
    CERTIFIED = "CERTIFIED"
    PENDING = "PENDING"
    REQUESTED = "REQUESTED"
    EXPIRED = "EXPIRED"


def generate_search_datacenter_filter(
    field,  # use Datacenter_Filter_Field_Enum
    value,
    is_not: bool = False,  # to handle exclusion
):
    return {
        "filterType": "term",
        "field": field,
        "value": value,
        "not": is_not,
    }

In [ ]:
generate_search_datacenter_filter(
    field=Datacenter_Filter_Field_Enum.CERTIFICATION.value,
    value=Datacenter_Filter_Field_Certification_Enum.CERTIFIED.value,
)

{'filterType': 'term',
 'field': 'certification.state',
 'value': 'CERTIFIED',
 'not': False}

In [ ]:
# | export


def generate_search_datacenter_filter_search_term(search_term):
    # if not "*" in search_term:
    #     search_term = f"*{search_term}*"

    return {"field": "name_sort", "filterType": "wildcard", "query": search_term}

##### sample implementation of generate_search_datacenter_filter_search_term


In [ ]:
generate_search_datacenter_filter_search_term("hello world*")

{'field': 'name_sort', 'filterType': 'wildcard', 'query': 'hello world*'}

In [ ]:
# | export


def generate_search_datacenter_body(
    search_text: str = None,
    entity_type: Union[
        str, list
    ] = "DATASET",  # can accept one entity_type or a list of entity_types
    additional_filters_ls: list[dict] = None,
    combineResults: bool = True,
    limit: int = 100,
    offset: int = 0,
):
    filters_ls = (
        [generate_search_datacenter_filter_search_term(search_text)]
        if search_text
        else []
    )

    if not isinstance(entity_type, list):
        entity_type = [entity_type]

    if additional_filters_ls:
        if not isinstance(additional_filters_ls, list):
            additional_filters_ls = [additional_filters_ls]

        filters_ls += additional_filters_ls

    return {
        "entities": entity_type,
        "filters": filters_ls or [],
        "combineResults": combineResults,
        "query": "*",
        "count": limit,
        "offset": offset,
    }

#### sample implementation of generate_search_datacenter_body


In [ ]:
from pprint import pprint

sample_search_body = generate_search_datacenter_body(
    search_text="hello world",
    entity_type=Datacenter_Enum.DATAFLOW.value,
    additional_filters_ls=[Dataflow_Type_Filter_Enum.MAGICV2.value],
)

pprint(sample_search_body)

{'combineResults': True,
 'count': 100,
 'entities': ['DATAFLOW'],
 'filters': [{'field': 'name_sort',
              'filterType': 'wildcard',
              'query': 'hello world'},
             {'field': 'data_flow_type',
              'filterType': 'term',
              'name': 'Magic ETL v2',
              'not': False,
              'value': 'MAGIC'}],
 'offset': 0,
 'query': '*'}


In [ ]:
# | export
def generate_search_datacenter_account_body(
    search_str: str, is_exact_match: bool = True
):
    return {
        # "count": 100,
        # "offset": 0,
        "combineResults": False,
        "query": search_str if is_exact_match else f"*{search_str}*",
        "filters": [],
        "facetValuesToInclude": [
            "DATAPROVIDERNAME",
            "OWNED_BY_ID",
            "VALID",
            "USED",
            "LAST_MODIFIED_DATE",
        ],
        "queryProfile": "GLOBAL",
        "entityList": [["account"]],
        "sort": {"fieldSorts": [{"field": "display_name_sort", "sortOrder": "ASC"}]},
    }

In [ ]:
# | export
class SearchDatacenter_NoResultsFound(de.DomoError):
    def __init__(
        self, body, domo_instance, parent_class: str = None, function_name: str = None
    ):
        super().__init__(
            message=body,
            domo_instance=domo_instance,
            parent_class=parent_class,
            function_name=function_name,
        )


@gd.route_function
async def search_datacenter(
    auth: dmda.DomoAuth,
    maximum: int = None,
    body: dict = None,  # either pass a body or generate a body in the function using search_text, entity_type, and additional_filters parameters
    search_text=None,
    entity_type: Union[
        str, list
    ] = "dataset",  # can accept one value or a list of values
    additional_filters_ls=None,
    arr_fn: callable = None,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    debug_loop: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1,
) -> rgd.ResponseGetData:
    limit = 100  # api enforced limit

    if not body:
        body = generate_search_datacenter_body(
            entity_type=entity_type,
            additional_filters_ls=additional_filters_ls,
            search_text=search_text,
            combineResults=False,
            limit=limit,
        )

    if not arr_fn:

        def arr_fn(res):
            return res.response.get("searchObjects")

    url = f"https://{auth.domo_instance}.domo.com/api/search/v1/query"

    res = await gd.looper(
        auth=auth,
        session=session,
        url=url,
        loop_until_end=True if not maximum else False,
        body=body,
        offset_params_in_body=True,
        offset_params={"offset": "offset", "limit": "count"},
        arr_fn=arr_fn,
        method="POST",
        maximum=maximum,
        limit=limit,
        debug_api=debug_api,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        debug_loop=debug_loop,
    )

    if res.is_success and len(res.response) == 0:
        raise SearchDatacenter_NoResultsFound(
            body=body,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    return res

#### sample implementation of generate_search body and search datacenter


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


additional_filters_ls = [
    generate_search_datacenter_filter("dataprovidername_facet", "Jupyter Data")
]


res = await search_datacenter(
    auth=token_auth,
    search_text="*kb*",
    entity_type=Datacenter_Enum.DATASET.value,
    additional_filters_ls=additional_filters_ls,
    # debug_api = False,
    # maximum = 10
)

pd.DataFrame(res.response)

,entityType,databaseId,searchId,createDate,lastModified,lastIndexed,highlightedFields,language,requestAccess,score,...,pdpEnabled,used,owners,ownersLocalized,cloudName,cloudId,winnerText,systemTagFields,ownedByType,customer
0,dataset,04c1574e-c8be-4721-9846-c6ffa491144b,"{'indexName': None, 'databaseId': '04c1574e-c8...",1668379680000,1699634088000,1699634766455,{},English,False,11.0,...,True,False,"[{'id': '1893952720', 'type': 'USER', 'display...","{'localizedMessage': 'Jae Wilson1', 'count': 1}",Domo,domo,domo_kbs,[system_tags],USER,mmmm-0012-0200


In [ ]:
# | export
@gd.route_function
async def get_lineage_upstream(
    auth: dmda.DomoAuth,
    entity_type: str,
    entity_id: str,
    session: httpx.AsyncClient = None,
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
):
    url = f"https://{auth.domo_instance}.domo.com/api/data/v1/lineage/{entity_type}/{entity_id}"

    params = {"traverseDown": "false"}

    return await gd.get_data(
        auth=auth,
        method="GET",
        url=url,
        params=params,
        session=session,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

#### sample implementation of get_lineage_upstream


In [ ]:
import os
import pandas as pd

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_lineage_upstream(
    auth=token_auth, entity_type=Datacenter_Enum.DATAFLOW.value, entity_id=4
)

# pd.DataFrame([obj for obj in res.response.values()])

res.response

{'DATA_SOURCE45a7a24e-c738-44f9-9019-00d5a33613ae': {'type': 'DATA_SOURCE',
  'id': '45a7a24e-c738-44f9-9019-00d5a33613ae',
  'descendantCounts': {},
  'ancestorCounts': {'DATA_SOURCE': 2, 'DATAFLOW': 2},
  'complete': True,
  'children': [{'type': 'DATAFLOW',
    'id': '4',
    'complete': True,
    'children': [],
    'parents': []}],
  'parents': [{'type': 'DATAFLOW',
    'id': '3',
    'complete': True,
    'children': [],
    'parents': []}]},
 'DATAFLOW4': {'type': 'DATAFLOW',
  'id': '4',
  'descendantCounts': {},
  'ancestorCounts': {'DATA_SOURCE': 3, 'DATAFLOW': 2},
  'complete': True,
  'children': [],
  'parents': [{'type': 'DATA_SOURCE',
    'id': '45a7a24e-c738-44f9-9019-00d5a33613ae',
    'complete': True,
    'children': [],
    'parents': []}]},
 'DATA_SOURCE0647a9ed-1a8e-420e-bd66-34f313a18595': {'type': 'DATA_SOURCE',
  'id': '0647a9ed-1a8e-420e-bd66-34f313a18595',
  'descendantCounts': {},
  'ancestorCounts': {'DATA_SOURCE': 1, 'DATAFLOW': 1},
  'complete': True,
  '

In [ ]:
# | export
class ShareResource_Enum(Enum):
    PAGE = "page"
    CARD = "badge"


@gd.route_function
async def share_resource(
    auth: dmda.DomoAuth,
    resource_ids: list,
    resource_type: ShareResource_Enum,
    group_ids: list = None,
    user_ids: list = None,
    message: str = None,  # email to user
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
):
    """shares page or card with users or groups

    body format:  {
        "resources": [
            {
                "type": "page",
                "id": {oage_id}
            }
        ],
        "recipients": [
            {
                "type": "group",
                "id": "{group_id}"
            }
        ],
        "message": "I thought you might find this page interesting."
    }"""

    resource_ids = resource_ids if isinstance(resource_ids, list) else [resource_ids]
    if group_ids:
        group_ids = (
            group_ids and group_ids if isinstance(group_ids, list) else [group_ids]
        )

    if user_ids:
        user_ids = user_ids if isinstance(user_ids, list) else [user_ids]

    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/share?sendEmail=false"

    recipient_ls = []

    if group_ids:
        [recipient_ls.append({"type": "group", "id": str(id)}) for id in group_ids]

    if user_ids:
        [recipient_ls.append({"type": "user", "id": str(id)}) for id in user_ids]

    resource_ls = [{"type": resource_type.value, "id": str(id)} for id in resource_ids]

    body = {
        "resources": resource_ls,
        "recipients": recipient_ls,
        "message": message,
    }

    res = await gd.get_data(
        url,
        method="POST",
        auth=auth,
        body=body,
        session=session,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if res.is_success:
        res.response = f"{resource_type.value} {','.join([resource['id'] for resource in  resource_ls])} successfully shared with {', '.join([recipient['id'] for recipient in recipient_ls])}"

    return res

#### sample implementation of share_content

In [ ]:
import os
import domolibrary.client.DomoAuth as dmda


token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)


await share_resource(
    auth=token_auth,
    resource_ids=[1601825647],
    resource_type=ShareResource_Enum.CARD,
    group_ids=[1324037627],
    message=None,
    debug_api=False,
)

ResponseGetData(status=200, response='badge 1601825647 successfully shared with 1324037627', is_success=True, parent_class=None)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

!nbqa black datacenter.ipynb

All done! ✨ 🍰 ✨
1 file left unchanged.
